In [ ]:
ls

In [1]:
import os
import re
import pandas as pd
import pprint
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import nltk  
# nltk.download("wordnet")  # only done once! we have to download the WordNet database locally
from nltk.tokenize import TreebankWordTokenizer # very good tokenizer for english, considers sentence structure
from nltk.stem import WordNetLemmatizer

In [ ]:
# r = requests.get('https://www.lyrics.com/artist/2Pac/50051')
# # r = r.text

In [ ]:
URL_artist = 'https://www.lyrics.com/artist/2Pac/50051'
# URL_artist = 'https://www.lyrics.com/artist/The-Notorious-B.I.G./44889'
page_artist = requests.get(URL_artist)
soup_artist = BeautifulSoup(page_artist.content, "html.parser")

In [ ]:
song_albums = []
album_names = []

for link in soup_artist.find_all(href=re.compile('/album')):
    song_albums.append(link.get('href'))
    album_names.append(link.get_text())

In [ ]:
page_album = []
url = 0

for i in song_albums:
    page_album.append(requests.get(f'https://www.lyrics.com{song_albums[url]}'))
    url += 1

In [ ]:
soup_album = []
page = 0

for i in page_album:
    soup_album.append(BeautifulSoup(page_album[page].content, "html.parser"))
    page += 1

In [ ]:
lyrics = []

for link in soup_album:
    lyrics.append(link.find_all(href=re.compile('/lyric/[^"]+')))

In [ ]:
lyrics_href = []
lyrics_name = []
i = 0

for href in lyrics:
    for hrefi in href:
        lyrics_href.append(hrefi.get('href'))
        lyrics_name.append(hrefi.get_text())
# remove duplicate lyrics
lyrics_href = list(dict.fromkeys(lyrics_href))
lyrics_name = list(dict.fromkeys(lyrics_name))

lyrics_href = lyrics_href[:150]
lyrics_name = lyrics_name[:150]

In [ ]:
cd tupac/

In [ ]:
j = 0

for i in lyrics_href:
    URL_lyric = f'https://www.lyrics.com{i}'
    page_lyric = requests.get(URL_lyric)
    soup_lyric = BeautifulSoup(page_lyric.content, "html.parser")
    soup_lyric = soup_lyric.find(id="lyric-body-text")
    soup_lyric = soup_lyric.get_text()
    
    with open (f'Tupac_{lyrics_name[j]}.txt', 'w') as f:
        soup_lyric = f.write(soup_lyric)
        j += 1

In [2]:
cd tupac/

/Users/oguzhanbekar/OneDrive/Spiced/euclidean-eukalyptus-student-code/03_week/project/tupac


In [3]:
lyrics_text = []
artist_name = []

for fn in os.listdir():
    try:
        text = open(fn).read()
        lyrics_text.append(text.replace('\n', ' ').replace('\r', ''))
# .lower().replace('tupac','').replace('2pac','').replace('pac','').replace('biggie','').replace('big','').replace('b.i.g',''))
        artist_name.append(fn.split(sep='_')[0])
    except:
        pass
    
lyrics = pd.DataFrame({'artist':artist_name,'lyric':lyrics_text})

In [4]:
tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()
i=0

for doc in lyrics['lyric']:
    tokens = tokenizer.tokenize(text=doc)
    clean_doc = " ".join(lemmatizer.lemmatize(token) for token in tokens)
    lyrics['lyric'][i] = clean_doc
    i+=1

In [5]:
X_train,X_test,y_train,y_test = train_test_split(lyrics['lyric'], lyrics['artist'], test_size=0.2)

In [6]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((227,), (57,), (227,), (57,))

In [7]:
vectorizer = TfidfVectorizer(max_df=0.8,stop_words='english',lowercase=True)
X_train = vectorizer.fit_transform(X_train)
X_train_df = pd.DataFrame(X_train.todense(), columns=vectorizer.get_feature_names(), index=y_train)
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [8]:
clf.score(X_train,y_train)

0.9691629955947136

In [ ]:
# vectors_test2 = vectorizer.transform(Xtesting)
# predtesting = clf.predict(vectors_test2)
# # clf.score(vectors_test2,predtesting)
# predtesting

In [ ]:
ycontrol = pd.DataFrame({'test':y_test,'preds':pred,'controls':y_test == pred})

In [9]:
vectors_test = vectorizer.transform(X_test)
pred = clf.predict(vectors_test)
# pred,
clf.score(vectors_test,y_test)


0.6842105263157895